In [310]:
#------------------standard packages-----------------
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle
import json
from thefuzz import fuzz, process

#-----------------caio's modules---------------------
import sys, os #add code folder to sys.path
sys.path.append(os.path.abspath('../../data_processing'))

from cricsheet_read import cricsheet_read #data processing

#---------------------load data----------------------
match_stack, results = cricsheet_read()

#match_stack = match_stack[match_stack['league'] == 'Indian Premier League']

In [313]:
player_stack = match_stack.merge(results, left_on = 'match_id', right_on = results.index)

player_stack['set_team_win'] = (player_stack['innings'] == 1)*(player_stack['batting_team'] == player_stack['result']) + (
                               (player_stack['innings'] == 2)*(player_stack['bowling_team'] == player_stack['result']))

player_stack['set_team_toss'] = (player_stack['innings'] == 1)*(player_stack['batting_team'] == player_stack['toss_winner']) + (
                               (player_stack['innings'] == 2)*(player_stack['bowling_team'] == player_stack['toss_winner']))

bat_stack = player_stack[['match_id', 'start_date', 'innings', 'striker', 'set_team_win', 'set_team_toss',
                           ]].rename(columns = {'batting_team':'team', 'striker':'player'})

bowl_stack = player_stack[['match_id', 'start_date', 'innings', 'bowler','set_team_win', 'set_team_toss',
                          ]].rename(columns = {'bowling_team':'team', 'bowler':'player'})

player_stack = pd.concat([bat_stack,bowl_stack]).drop_duplicates(['match_id', 'player'])
player_stack['days_since_match'] = (dt.datetime.today() - player_stack['start_date']).dt.days

info_stack =  player_stack[['match_id', 'days_since_match', 'set_team_win', 'set_team_toss']].drop_duplicates()
player_stack = player_stack[['match_id', 'innings', 'player']]

appearances = player_stack.groupby('player', as_index = False).count()[
                                  ['player', 'match_id']].rename(columns = {'match_id':'n_matches'})

player_stack = player_stack.merge(appearances, on = 'player')

player_stack = player_stack[player_stack['n_matches'] > 7].drop('n_matches', axis = 1)

player_stack = player_stack.pivot(index = 'match_id',
                                  columns = 'player', values = 'innings').fillna(0)#.rename_axis(columns = None)

player_stack = player_stack.merge(info_stack, left_on = player_stack.index, right_on = 'match_id')

player_stack = player_stack.sort_values(['days_since_match','match_id']).reset_index(drop = True)

player_stack.to_csv('../../../data/player_data.csv', index = False)